In [22]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np # linear algebra
import os
# !pip install ltp
# from ltp import LTP

#!pip install nxviz
#!pip install netgraph

!git clone https://github.com/jinfeijoy/graph-analytics

import jieba
data_path = "/content/graph-analytics/social_network_analysis/他山之石HongLou/data"
jieba.load_userdict(os.path.join(data_path,"honglou_name_addtional_dic.txt"))


fatal: destination path 'graph-analytics' already exists and is not an empty directory.


In [2]:
import re

def __merge_symmetry(sentences, symmetry=('“','”')):
    '''合并对称符号，如双引号'''
    effective_ = []
    merged = True
    for index in range(len(sentences)):       
        if symmetry[0] in sentences[index] and symmetry[1] not in sentences[index]:
            merged = False
            effective_.append(sentences[index])
        elif symmetry[1] in sentences[index] and not merged:
            merged = True
            effective_[-1] += sentences[index]
        elif symmetry[0] not in sentences[index] and symmetry[1] not in sentences[index] and not merged :
            effective_[-1] += sentences[index]
        else:
            effective_.append(sentences[index])
        
    return [i.strip() for i in effective_ if len(i.strip()) > 0]

def to_sentences(paragraph):
    """cut paragraph to sentence"""
    sentences = re.split(r"(？|。|！|\…\…)", paragraph)
    sentences.append("")
    sentences = ["".join(i) for i in zip(sentences[0::2], sentences[1::2])]
    sentences = [i.strip() for i in sentences if len(i.strip()) > 0]
    
    for j in range(1, len(sentences)):
        if sentences[j][0] == '”':
            sentences[j-1] = sentences[j-1] + '”'
            sentences[j] = sentences[j][1:]
            
    return __merge_symmetry(sentences)

In [23]:
import re
with open(os.path.join(data_path,'红楼梦白话文.txt'),"r", encoding="utf8") as f:
    text = f.read()

text = re.sub('\s', '', text)

In [4]:
sentence_raw = to_sentences(text)
sentence_raw[200:210]

['正说着，只听后院中笑声朗朗，有人说：“我来迟了，不曾迎接远客！”',
 '黛玉暗想，这里人人敛气屏声，是谁如此放诞无礼？',
 '只见一群媳妇丫鬟拥着一位恍若天仙般的盛装丽人走进来。',
 '贾母笑着说：“她是我们这里有名的泼辣货，就是南京说的‘辣子’，你只叫她‘凤辣子’就是了。”',
 '众姊妹告诉黛玉：“这是琏嫂子。”',
 '黛玉想起母亲说过，大舅贾赦的儿子贾琏，娶的是二舅母王氏的娘家侄女，自幼男孩儿般教养，名叫王熙凤，忙笑着见了礼，称呼“嫂子”。',
 '熙凤拉着黛玉的手，仔细打量了一阵，送到贾母身边坐下，笑着说：“天下真有这样标致的人物，我今天才算见了！看她那气派，竟不像老祖宗的外孙女，是个嫡亲的孙女，怨不得老祖宗天天挂在心上。只可怜我妹妹这么命苦，怎么姑妈偏偏去世了！”',
 '边说边用手帕擦泪。',
 '贾母笑着说：“我才好了，你又来招我。你妹妹远路才来，身子又弱，快别再说这些话。”',
 '熙凤一面自责“该打”，一面一连声地问黛玉：几岁了？']

In [60]:
clean_name = pd.read_csv(os.path.join(data_path,'name_clean_sorted.csv'))[['文中名字','标准姓名']]
clean_name['标准姓名'] = np.where(clean_name['标准姓名'].isna(), clean_name['文中名字'], clean_name['标准姓名'])
clean_name.sample(3)

,文中名字,标准姓名
104,李嬷嬷,李嬷嬷
112,琏,贾琏
189,田家,田家


In [62]:
clean_name_dict = dict(clean_name.values)
clean_name_dict

{'丁郎': '丁郎',
 '严老爷': '严老爷',
 '丰儿': '丰儿',
 '临安伯': '临安伯',
 '乌木做': '乌木做',
 '云儿': '云儿',
 '云光': '云光',
 '京兆尹': '京兆尹',
 '代儒': '代儒',
 '何三': '何三',
 '佳蕙': '佳蕙',
 '倪二': '倪二',
 '倪二娘子': '倪二娘子',
 '元春': '贾元春',
 '兰哥儿': '贾兰',
 '冯公子': '冯公子',
 '冯唐': '冯唐',
 '冯将军': '冯将军',
 '冯渊': '冯渊',
 '冯紫英': '冯紫英',
 '冯胖子': '冯胖子',
 '冷子兴': '冷子兴',
 '凤丫头': '王熙凤',
 '凤哥儿': '王熙凤',
 '凤姐': '王熙凤',
 '凤姐儿': '王熙凤',
 '凤姐姐': '王熙凤',
 '凤姑娘': '王熙凤',
 '刘亲家': '刘亲家',
 '刘姥姥': '刘姥姥',
 '刘氏': '刘氏',
 '刘铁嘴': '刘铁嘴',
 '包勇': '包勇',
 '北静王': '北静王',
 '单聘仁': '单聘仁',
 '卜固修': '卜固修',
 '占花魁': '占花魁',
 '史侯': '史侯',
 '史大妹妹': '史湘云',
 '史大姑娘': '史湘云',
 '史姑娘': '史湘云',
 '史湘云': '史湘云',
 '史老太君': '史老太君',
 '叶妈': '叶妈',
 '司棋': '司棋',
 '司马相如': '司马相如',
 '向宁': '向宁',
 '向宝玉': '向宝玉',
 '吴兴家': '吴兴家',
 '吴巡抚': '吴巡抚',
 '吴新登': '吴新登',
 '吴贵': '吴贵',
 '周大娘': '周大娘',
 '周姨娘': '周姨娘',
 '周母': '周母',
 '周瑞': '周瑞',
 '周管家': '周管家',
 '唐伯虎': '唐伯虎',
 '唐多令': '唐多令',
 '四儿': '四儿',
 '圆信': '圆信',
 '士隐': '甄士隐',
 '夏太监': '夏太监',
 '夏妈': '夏妈',
 '夏子': '夏子',
 '夏母': '夏母',
 '夏秉忠': '夏秉忠',
 '妙公': '妙公',
 '妙玉': '妙玉',
 '子兴': '子兴',


In [27]:
my_file = open(os.path.join(data_path,"honglou_name_addtional_dic.txt"), "r")
content = my_file.read()
content_list = content.split("\n")
my_file.close()
content_list[0:3]

['艾官', '白海棠', '板儿']

In [65]:
clean_name_dict['黛玉']

'林黛玉'

In [66]:
from itertools import combinations

sentence = sentence_raw[206]
seg_list = jieba.cut(sentence, cut_all=True)
[i for i in combinations([clean_name_dict[i] for i in list(seg_list) if i in content_list],2)]

[('王熙凤', '林黛玉'), ('王熙凤', '贾母'), ('林黛玉', '贾母')]

In [68]:
characters_comb = []
for i in range(len(sentence_raw)):
  sentence = sentence_raw[i]
  seg_list = jieba.cut(sentence, cut_all=True)
  characters_comb.extend([item for item in combinations([clean_name_dict[item] for item in list(seg_list) if item in content_list],2)])

characters_comb

KeyError: ignored

In [45]:
len(sentence_raw)

6479